In [1]:
import requests
import re
import json
import pandas as pd

In [2]:
from scrape_house_price_links_ import country_links

print(len(country_links))

Links encontrados: 57
https://www.theglobaleconomy.com/Australia/house_price_index/
https://www.theglobaleconomy.com/Austria/house_price_index/
https://www.theglobaleconomy.com/Belgium/house_price_index/
https://www.theglobaleconomy.com/Brazil/house_price_index/
https://www.theglobaleconomy.com/Bulgaria/house_price_index/
https://www.theglobaleconomy.com/Canada/house_price_index/
https://www.theglobaleconomy.com/Chile/house_price_index/
https://www.theglobaleconomy.com/China/house_price_index/
https://www.theglobaleconomy.com/Colombia/house_price_index/
https://www.theglobaleconomy.com/Croatia/house_price_index/
https://www.theglobaleconomy.com/Cyprus/house_price_index/
https://www.theglobaleconomy.com/Czech-Republic/house_price_index/
https://www.theglobaleconomy.com/Denmark/house_price_index/
https://www.theglobaleconomy.com/Estonia/house_price_index/
https://www.theglobaleconomy.com/Finland/house_price_index/
https://www.theglobaleconomy.com/France/house_price_index/
https://www.the

In [6]:
from urllib.parse import urlparse

# DataFrame maestro
master_df = pd.DataFrame()

pattern = r'arrayToDataTable\((\[\[.*?\]\])\)'

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"
}


for i in range(3):
    try:
        # Obtener HTML
        url = country_links[i]
        response = requests.get(url, headers=headers)
        html = response.text

        # Buscar los datos con regex
        match = re.search(pattern, html, re.DOTALL)
        if match:
            data_str = match.group(1)
            data = json.loads(data_str)

            # Convertir a DataFrame
            df = pd.DataFrame(data[1:], columns=data[0])  # data[0] = headers
            df.columns = ["Period", "Value"]

            # Extraer nombre del país desde el URL
            country = urlparse(url).path.strip("/").split("/")[0]

            # Agregar columna con el país
            df["Country"] = country

            # Convertir a estructura pivotada
            df_pivot = df.pivot(index="Period", columns="Country", values="Value")

            # Unir con el DataFrame maestro
            master_df = pd.merge(master_df, df_pivot, how="outer", left_index=True, right_index=True)

            print(f"✅ Datos extraídos correctamente para {country}")
        else:
            print(f"⚠️ No se encontraron datos en {url}")
    except Exception as e:
        print(f"❌ Error procesando {url}: {e}")

✅ Datos extraídos correctamente para Australia
✅ Datos extraídos correctamente para Austria
✅ Datos extraídos correctamente para Belgium


In [11]:
# Resetear el índice para separar "Period"
master_df = master_df.reset_index()

In [12]:
# Separar "Q1 1990" en "Quarter" y "Year"
master_df[["Quarter", "Year"]] = master_df["Period"].str.extract(r"Q(\d)\s+(\d{4})").astype(int)

In [13]:

# Reordenar las columnas: primero Quarter y Year
cols = ["Quarter", "Year"] + [col for col in master_df.columns if col not in ["Period", "Quarter", "Year"]]
master_df = master_df[cols]

In [14]:
# Ordenar por Year y Quarter
master_df = master_df.sort_values(by=["Year", "Quarter"]).reset_index(drop=True)

In [15]:
master_df.head()

,Quarter,Year,Australia,Austria,Belgium
0,1,1990,28.24,51.86,32.75
1,2,1990,28.50,58.38,32.67
2,3,1990,28.24,64.82,33.63
3,4,1990,28.41,65.45,34.01
4,1,1991,28.38,67.18,34.21


In [16]:
master_df.tail()

,Quarter,Year,Australia,Austria,Belgium
136,1,2024,188.10,208.05,152.14
137,2,2024,192.13,207.82,151.71
138,3,2024,194.75,207.50,154.11
139,4,2024,195.13,207.11,155.29
140,1,2025,NaN,208.84,NaN
